СКР

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

TRAIN_FILE = "train_СКР.xlsx"
ACTUAL_FILE = "СКР_2024.xlsx"

# Функция очистки чисел с запятых и пробелов
def clean_numeric(series):
    return (
        series.astype(str)
        .str.replace('\xa0', '', regex=False)
        .str.replace(' ', '')
        .str.replace(',', '.')
        .astype(float)
    )

# Функция для стандартизации названий регионов
def standardize_region_names(df):
    """Стандартизация названий регионов"""
    df_clean = df.copy()

    replacements = {
        'Ненецкий авт.округ': 'Ненецкий автономный округ',
        'Hенецкий авт.округ': 'Ненецкий автономный округ',
        '  Ненецкий автономный округ': 'Ненецкий автономный округ',
        'Ямало-Ненецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        'Ямало-Hенецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        '  Ямало-Ненецкий автономный округ': 'Ямало-Ненецкий автономный округ',
        'Ханты-Мансийский авт.округ-Югра': 'Ханты-Мансийский автономный округ - Югра',
        '  Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский автономный округ - Югра',
        'Республика Татарстан(Татарстан)': 'Республика Татарстан',
        'Чувашская Республика(Чувашия)': 'Чувашская Республика',
        'Республика Северная Осетия- Алания': 'Республика Северная Осетия-Алания',
        'Oмская область': 'Омская область',
        'Hижегородская область': 'Нижегородская область',
        'г. Севастополь': 'г.Севастополь',
        'г.Москва': 'г.Москва',
        'г.Санкт-Петербург': 'г.Санкт-Петербург',
        'Чукотский авт.округ': 'Чукотский автономный округ',
    }

    df_clean['Регион'] = df_clean['Регион'].replace(replacements)
    df_clean['Регион'] = df_clean['Регион'].str.strip()
    return df_clean

# Загрузка данных
df_train = pd.read_excel(TRAIN_FILE)
df_actual = pd.read_excel(ACTUAL_FILE)

# Применяем стандартизацию
print("Стандартизация названий регионов...")
df_train = standardize_region_names(df_train)
df_actual = standardize_region_names(df_actual)

# Очистка СКР
df_train['СКР'] = clean_numeric(df_train['СКР'])
df_actual['СКР'] = clean_numeric(df_actual['СКР'])

# Убедимся, что годы — целые числа
df_train['Год'] = pd.to_numeric(df_train['Год'], errors='coerce')
df_actual['Год'] = pd.to_numeric(df_actual['Год'], errors='coerce')

# Берём только 2014–2023
df_train = df_train[(df_train['Год'] >= 2014) & (df_train['Год'] <= 2023)]

# Создаём словарь: регион - фактический СКР за 2024
actual_dict = dict(zip(df_actual['Регион'], df_actual['СКР']))

# Общие регионы
common_regions = set(df_train['Регион']) & set(df_actual['Регион'])
print(f"Общих регионов между датасетами: {len(common_regions)}")

results = []

for region in sorted(common_regions):
    ts_data = df_train[df_train['Регион'] == region].sort_values('Год')
    if len(ts_data) < 10:
        print(f"Пропущен {region}: мало данных ({len(ts_data)} лет)")
        continue

    scr_values = ts_data['СКР'].values
    actual_2024 = actual_dict[region]

    best_forecast = np.nan
    best_aic = np.inf

    # Перебираем небольшой набор параметров ARIMA
    for p in range(0, 3):
        for d in range(0, 2):
            for q in range(0, 3):
                try:
                    model = ARIMA(scr_values, order=(p, d, q))
                    fitted = model.fit()
                    forecast = fitted.forecast(steps=1)[0]
                    if fitted.aic < best_aic:
                        best_aic = fitted.aic
                        best_forecast = forecast
                except:
                    continue

    # Fallback
    if pd.isna(best_forecast):
        try:
            fallback = ARIMA(scr_values, order=(1,1,1)).fit()
            best_forecast = fallback.forecast(steps=1)[0]
        except:
            print(f"Не удалось построить модель для {region}")
            continue

    # Метрики
    abs_error = abs(best_forecast - actual_2024)
    rmse = np.sqrt(mean_squared_error([actual_2024], [best_forecast]))
    mae = mean_absolute_error([actual_2024], [best_forecast])

    results.append({
        'Регион': region,
        'Прогноз_СКР_2024': round(best_forecast, 4),
        'Факт_СКР_2024': round(actual_2024, 4),
        'Абсолютная_ошибка': round(abs_error, 4),
        'RMSE': round(rmse, 4),
        'MAE': round(mae, 4)
    })

    print(f"{region}: прогноз = {best_forecast:.4f}, факт = {actual_2024:.4f}, ошибка = {abs_error:.4f}")

# Итоги
if results:
    results_df = pd.DataFrame(results)
    overall_rmse = np.sqrt(mean_squared_error(results_df['Факт_СКР_2024'], results_df['Прогноз_СКР_2024']))
    overall_mae = mean_absolute_error(results_df['Факт_СКР_2024'], results_df['Прогноз_СКР_2024'])

    print("\n" + "="*60)
    print(f"Общий RMSE по СКР: {overall_rmse:.4f}")
    print(f"Общий MAE по СКР: {overall_mae:.4f}")
    print("="*60)

    # Сохраняем результат
    output_file = "прогноз_СКР_2024_результаты.xlsx"
    results_df.to_excel(output_file, index=False)

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

FILE_PATH = "train_ОПЖ.xlsx"

# Функция очистки чисел
def clean_numeric(series):
    return (
        series.astype(str)
        .str.replace('\xa0', '', regex=False)
        .str.replace(' ', '')
        .str.replace(',', '.')
        .astype(float)
    )

# Функция стандартизации регионов
def standardize_region_names(df):
    df_clean = df.copy()
    replacements = {
        'Ненецкий авт.округ': 'Ненецкий автономный округ',
        'Hенецкий авт.округ': 'Ненецкий автономный округ',
        '  Ненецкий автономный округ': 'Ненецкий автономный округ',
        'Ямало-Ненецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        'Ямало-Hенецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        '  Ямало-Ненецкий автономный округ': 'Ямало-Ненецкий автономный округ',
        'Ханты-Мансийский авт.округ-Югра': 'Ханты-Мансийский автономный округ - Югра',
        '  Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский автономный округ - Югра',
        'Республика Татарстан(Татарстан)': 'Республика Татарстан',
        'Чувашская Республика(Чувашия)': 'Чувашская Республика',
        'Республика Северная Осетия- Алания': 'Республика Северная Осетия-Алания',
        'Oмская область': 'Омская область',
        'Hижегородская область': 'Нижегородская область',
        'г. Севастополь': 'г.Севастополь',
        'г.Москва': 'г.Москва',
        'г.Санкт-Петербург': 'г.Санкт-Петербург',
        'Чукотский авт.округ': 'Чукотский автономный округ',
    }
    df_clean['Регион'] = df_clean['Регион'].replace(replacements)
    df_clean['Регион'] = df_clean['Регион'].str.strip()
    return df_clean

# Загрузка данных
df = pd.read_excel(FILE_PATH)

# Стандартизация и очистка
df = standardize_region_names(df)
df['ОПЖ'] = clean_numeric(df['ОПЖ'])
df['Год'] = pd.to_numeric(df['Год'], errors='coerce').astype(int)
df = df[(df['Год'] >= 2014) & (df['Год'] <= 2023)]

# Группируем по регионам
regions = df['Регион'].unique()
results = []

for region in sorted(regions):
    ts = df[df['Регион'] == region].sort_values('Год')
    if len(ts) < 10:
        print(f"{region}: мало данных ({len(ts)} лет)")
        continue

    opzh_vals = ts['ОПЖ'].values
    train = opzh_vals[:-1]
    actual_2023 = opzh_vals[-1]

    # Подбор лучшей ARIMA на train
    best_forecast = np.nan
    for p in range(0, 3):
        for d in range(0, 2):
            for q in range(0, 3):
                try:
                    model = ARIMA(train, order=(p, d, q))
                    fitted = model.fit()
                    forecast = fitted.forecast(steps=1)[0]
                    if np.isfinite(forecast):
                        best_forecast = forecast
                        break
                except:
                    continue
            if not pd.isna(best_forecast):
                break

    # Fallback
    if pd.isna(best_forecast):
        try:
            model = ARIMA(train, order=(1,1,1)).fit()
            best_forecast = model.forecast(steps=1)[0]
        except:
            print(f"Не удалось для {region}")
            continue

    # Метрики
    rmse = np.sqrt(mean_squared_error([actual_2023], [best_forecast]))
    mae = mean_absolute_error([actual_2023], [best_forecast])

    results.append({
        'Регион': region,
        'Прогноз_ОПЖ_2023': round(best_forecast, 4),
        'Факт_ОПЖ_2023': round(actual_2023, 4),
        'Абсолютная_ошибка': round(abs(best_forecast - actual_2023), 4),
        'RMSE': round(rmse, 4),
        'MAE': round(mae, 4)
    })

    print(f"{region}: прогноз = {best_forecast:.4f}, факт = {actual_2023:.4f}, ошибка = {abs(best_forecast - actual_2023):.4f}")

# Сводные метрики
if results:
    results_df = pd.DataFrame(results)
    overall_rmse = np.sqrt(mean_squared_error(results_df['Факт_ОПЖ_2023'], results_df['Прогноз_ОПЖ_2023']))
    overall_mae = mean_absolute_error(results_df['Факт_ОПЖ_2023'], results_df['Прогноз_ОПЖ_2023'])

    print("\n" + "="*60)
    print(f"Общий RMSE по ОПЖ: {overall_rmse:.4f}")
    print(f"Общий MAE по ОПЖ: {overall_mae:.4f}")
    print("="*60)

    # Сохраняем результат
    output_file = "валидация_ОПЖ_прогноз_2023.xlsx"
    results_df.to_excel(output_file, index=False)

    from google.colab import files
    files.download(output_file)
else:
    print("Нет данных для обработки.")